In [35]:
import pickle as p
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from tqdm import tqdm
import scipy.signal as sig
from sklearn.linear_model import LogisticRegression
from scipy.interpolate import interp1d
from scipy.integrate import romb
plt.rcParams.update({'font.size': 22})
import glob
from scipy.signal import medfilt,butter
import scipy
import math

In [2]:
n_splits = 10

def num_to_epoch(epoch_number):
    if epoch_number == 0:
        return 'UP1'
    elif epoch_number == 1:
        return 'UP2'
    elif epoch_number == 2:
        return 'P1'
    elif epoch_number == 3:
        return 'P2'
    elif epoch_number == 4:
        return 'REC'
    else:
        raise Exception('Invalid Epoch Number')

def num_to_subject(subject_number):
    return 'Subject'+str(subject_number)

def get_signal(measure, epoch, subjno):
    if isinstance(epoch,str):
        return np.ravel(df.loc[measure, epoch][num_to_subject(subjno)])
    elif isinstance(epoch, int):
        return np.ravel(df.loc[measure, num_to_epoch(epoch)][num_to_subject(subjno)])

def get_split_signal(measure, epoch, subjno, splitno):
    if isinstance(epoch,str):
        return np.array_split(np.ravel(df.loc[measure, epoch][num_to_subject(subjno)]), n_splits)[splitno]
    elif isinstance(epoch, int):
        return np.array_split(np.ravel(df.loc[measure, num_to_epoch(epoch)][num_to_subject(subjno)]),n_splits)[splitno]
    
def full_signal(measure, subjno):
    return np.concatenate((
        np.ravel(df.loc[measure, num_to_epoch(0)][num_to_subject(subjno)]),
        np.ravel(df.loc[measure, num_to_epoch(1)][num_to_subject(subjno)]),
        np.ravel(df.loc[measure, num_to_epoch(2)][num_to_subject(subjno)]),
        np.ravel(df.loc[measure, num_to_epoch(3)][num_to_subject(subjno)]),
        np.ravel(df.loc[measure, num_to_epoch(4)][num_to_subject(subjno)]),
    ))

def is_perturbed(epoch):
    if epoch == 0 or epoch == 1 or epoch == 4:
        return 0
    elif epoch == 2 or epoch == 3:
        return 1
    
def scaled_correlation_time(signal1, signal2):
    signal1 = (signal1 - np.mean(signal1))/np.std(signal1)
    signal2 = (signal2 - np.mean(signal2))/np.std(signal2)
    acorr = np.correlate(signal1, signal2, mode='full')
    acorr = acorr[(acorr.size // 2 ):] / np.max(acorr)
#     plt.plot(acorr)
    tau = np.argmax([acorr < 1/np.exp(1)])
    return tau / len(acorr)

def correlation_integral(signal1, signal2):
    signal1 = (signal1 - np.mean(signal1))/np.std(signal1)
    signal2 = (signal2 - np.mean(signal2))/np.std(signal2)
    acorr = np.correlate(signal1, signal2, mode='full')
    acorr = acorr[(acorr.size // 2 ):] / np.max(acorr)
#     plt.plot(acorr)
    integral = np.trapz(acorr)
    return integral

def plot_full_experiment(measure, subjno):
    s1 = np.ravel(df.loc[measure, num_to_epoch(0)][num_to_subject(subjno)])
    s2 = np.ravel(df.loc[measure, num_to_epoch(1)][num_to_subject(subjno)])
    s3 = np.ravel(df.loc[measure, num_to_epoch(2)][num_to_subject(subjno)])
    s4 = np.ravel(df.loc[measure, num_to_epoch(3)][num_to_subject(subjno)])
    s5 = np.ravel(df.loc[measure, num_to_epoch(4)][num_to_subject(subjno)])
    fullsignal = np.concatenate((s1,s2,s3,s4,s5))
    plt.plot(fullsignal,'k',label = measure)
    plt.axvline(x = len(s1), color = 'k', linestyle = '--')
    plt.axvline(x = len(s1)+len(s2), color = 'k', linestyle = '--')
    plt.axvline(x = len(s1)+len(s2)+len(s3), color = 'k', linestyle = '--')
    plt.axvline(x = len(s1)+len(s2)+len(s3)+len(s4), color = 'k', linestyle = '--')
    plt.legend()

In [3]:
shimfiles = glob.glob('shimmerData/*/*')

In [4]:
def get_shimmer(subjno, part, epochno):
    epoch = num_to_epoch(epochno)
    if epoch == 'REC':
        epoch = 'Rec'
    elif epoch == 'P1':
        epoch = '_P1'
    elif epoch == 'P2':
        epoch = '_P2'
    
    for file in shimfiles:
        if str(subjno) in file and part in file and epoch in file:
            data = pd.read_csv(file, header = None)
    try:
        vectors = np.asarray(data[[1,2,3]])
    except:
        print('mising shimmer data. skipping...')
        return np.repeat(np.nan, 1000)
    mean = np.mean(vectors, axis = 0)
    std = np.std(vectors, axis = 0)
    
    vectors = (vectors - mean[np.newaxis,:])

    #plt.plot(vectors)
    
    z_vector = np.linalg.norm(vectors, axis = 1)

    return z_vector


In [5]:
def get_split_shimmer(subjno, part, epochno, splitno):

    z_vector = get_shimmer(subjno,part,epochno)

    return np.array_split(z_vector,n_splits)[splitno]


In [6]:
def signal_statistics(signal):
    mean = np.mean(signal)
    std = np.std(signal)
    skewness = stats.skew(signal)
    kurtosis = stats.kurtosis(signal)
    maximum = np.max(signal)
    minimum = np.min(signal)
    iqr = stats.iqr(signal)
    variation = stats.variation(signal)
    entropy = stats.entropy(np.abs(signal))
    corrtime = scaled_correlation_time(signal,signal)        
    return np.asarray([mean, std, skewness, kurtosis, maximum, minimum, iqr, variation, entropy, corrtime])

def spectrum_statistics(signal):
    
    fs,pxx = sig.periodogram(signal, fs = 50, nfft = 1000, scaling = 'density', detrend = 'constant')
    
#     plt.plot(fs,pxx)
    #plt.xlim(0,0.1)

    peak = fs[np.argmax(pxx)]
    peakmag = np.max(pxx)
    integral = np.trapz(pxx,fs)
    energy = np.dot(pxx,pxx)
    shannon = np.sum(pxx*np.log(1/pxx))

    # Add wavelet analysis

    return [peak, peakmag, integral, energy, shannon]

In [7]:
test_signal = get_shimmer(42, 'body', 3)


In [93]:
def max_dists(signal, m):
    count = 0
    N = len(signal)
    max_dists = []
    while count < N-m-1:
        x_i = signal[count:count+m-1]
        x_j = signal[count+1:count+m]
        max_dists.append(scipy.spatial.distance.chebyshev(x_i, x_j))
        count += 1
    return max_dists


def approx_entropy(signal):
    N = len(signal)
    m = 2 #or 3 -- dimensionality? 
    r = 0.3
        
    
    def phi(m):
        d_func = max_dists(signal,m)
        C = []
        for i in d_func:
            if i <= r and i > 0:
                C.append(i/(N-m+1)) 
        return (N - m + 1.0)**(-1) * sum(np.log10(C))
    return (abs(phi(m)-phi(m+1)))


approx_entropy(test_signal)


0.2766343281788908

In [94]:


def sample_entropy(signal):
    N = len(signal)
    m = 2
    r = 0.2
    
    A_list = max_dists(signal,m+1)
    B_list = max_dists(signal,m)
    
    count_A = 0
    count_B = 0
    for i in A_list:
        if i < r:
            count_A += 1
    for i in B_list: 
        if i < r:
            count_B += 1
    return math.log10(count_A/count_B)*-1

print(sample_entropy(test_signal))

0.009862868702608938


In [95]:
def multiscale_entropy(signal):
    mses = []
    for T in range(2,20,2):
        coarse_grain = []
        i = 0
        while i < len(signal):
            new_val = sum(signal[i:i+T])/T
            coarse_grain.append(new_val)
            i += T
        mses.append(sample_entropy(coarse_grain))
    return np.mean(mses)
        
print(multiscale_entropy(test_signal))



0.024643562693091963


In [14]:
bfeatures = [];
n_splits = 50
targets = [];
allfeatures = []
hfeatures = [];

for subjno in tqdm(all_subjects[8:]):
    for epoch in range(4):
        for splitno in range(n_splits):
            
            bshim_signal = get_split_shimmer(subjno,'body',epoch,splitno)
            bfeature = np.concatenate((signal_statistics(bshim_signal), spectrum_statistics(bshim_signal)))
            bfeatures.append(bfeature)
            
            hshim_signal = get_split_shimmer(subjno,'head',epoch,splitno)
            hfeature = np.concatenate((signal_statistics(hshim_signal), spectrum_statistics(hshim_signal)))
            hfeatures.append(hfeature)

            targets.append(is_perturbed(epoch))

            allfeature = np.concatenate((bfeature,hfeature))
            allfeatures.append(allfeature)
        

allfeature_names = ['bodyshim_mean', 'bodyshim_std', 'bodyshim_skewness', 'bodyshim_kurtosis', \
                    'bodyshim_maximum', 'bodyshim_minimum', \
                            'bodyshim_iqr', 'bodyshim_variation', 'bodyshim_entropy', 'bodyshim_corrtime',\
                       'bodyshim_peakfreq','bodyshim_peakpower','bodyshim_powerint','bodyshim_specenergy',\
                   'bodyshim_shannon', \
                   'headshim_mean', 'headshim_std', 'headshim_skewness', 'headshim_kurtosis', \
                    'headshim_maximum', 'headshim_minimum', \
                            'headshim_iqr', 'headshim_variation', 'headshim_entropy', 'headshim_corrtime',\
                       'headshim_peakfreq','headshim_peakpower','headshim_powerint','headshim_specenergy',\
                   'headshim_shannon']
fdf = pd.DataFrame(allfeatures, columns = allfeature_names)


NameError: name 'all_subjects' is not defined

In [15]:
targets = np.asarray(targets)
fdf = pd.DataFrame(allfeatures, columns = allfeature_names)


NameError: name 'allfeature_names' is not defined

In [16]:
from sklearn.feature_selection import mutual_info_classif
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc


In [17]:
where = np.asarray(np.sum(np.isnan(fdf),1)) > 0
X = np.asarray(fdf.loc[~where])
t = np.asarray(targets)[~where]


sc = preprocessing.MinMaxScaler()
X_scaled = sc.fit_transform(X)
scores = mutual_info_classif(X_scaled,t)


NameError: name 'fdf' is not defined

In [18]:
plt.figure(figsize=(15,5))
sortidx = np.argsort(-scores)
plt.bar(fdf.columns[sortidx],height=scores[sortidx],)
plt.xticks(rotation=45, ha='right',fontsize=15);
plt.title('Mutual Information Feature Selection')


NameError: name 'scores' is not defined

<Figure size 1080x360 with 0 Axes>